# Create a VM from an existing snapshot

Source: [Create a Windows VM from a specialized disk by using PowerShell](https://docs.microsoft.com/en-us/azure/virtual-machines/windows/create-vm-specialized#option-3-copy-an-existing-azure-vm)


Set the following variables and everything will be done for you

In [ ]:
$resourceGroupName = 'RG-ForVMs' 
$vmName = 'FromSnapshot'
$location = 'westus2' 
$snapshotName = 'Win10EntN21H2-snapshot'
$vmSize = "Standard_B2ms"
$rdpFilePath = "C:\Work\" + $vmName + ".rdp" 

If you want to explore your snapshots:

In [ ]:
Get-AzSnapshot -ResourceGroupName $resourceGroupName | Select Name

In [ ]:
#Get the snapshot
$snapshot = Get-AzSnapshot -ResourceGroupName $resourceGroupName -Name $snapshotName

In [ ]:
# Create the disk
$osDiskName = $vmName + "Disk"
$osDiskConfig = New-AzDiskConfig  -Location $location -CreateOption Copy -SourceResourceId $snapshot.Id
$osDisk = New-AzDisk -DiskName $osDiskName -Disk $osDiskConfig -ResourceGroupName $resourceGroupName

In [ ]:
# Create netwrok components
$subnetName = $vmName + 'SubNet'
$singleSubnet = New-AzVirtualNetworkSubnetConfig -Name $subnetName -AddressPrefix 10.0.0.0/24
$vnetName = $vmName + "VNet"
$vnet = New-AzVirtualNetwork -Name $vnetName -ResourceGroupName $resourceGroupName -Location $location -AddressPrefix 10.0.0.0/16 -Subnet $singleSubnet

$nsgName = $vmName +"Nsg"
$rdpRuleName = $vmName + "RdpRule"
$rdpRule = New-AzNetworkSecurityRuleConfig -Name $rdpRuleName -Description "Allow RDP" -Access ALlow -Protocol Tcp -Direction Inbound -Priority 110 -SourceAddressPrefix Internet -SourcePortRange * -DestinationAddressPrefix * -DestinationPortRange 3389
$nsg = New-AzNetworkSecurityGroup -ResourceGroupName $resourceGroupName -Location $location -Name $nsgName -SecurityRules $rdpRule

$ipName = $vmName + "IP"
$pip = New-AzPublicIpAddress -Name $ipName -ResourceGroupName $resourceGroupName -Location $location -AllocationMethod Dynamic

$nicName = $vmName + "NIC"
$nic = New-AzNetworkInterface -Name $nicName -ResourceGroupName $resourceGroupName -Location $location -SubnetId $vnet.Subnets[0].Id -PublicIpAddressId $pip.Id -NetworkSecurityGroupId $nsg.Id

In [ ]:

# Create VM config
$vmConfig = New-AzVMConfig -VMName $vmName -VMSize $vmSize
$vm = Add-AzVMNetworkInterface -VM $vmConfig -Id $nic.Id
$vm = Set-AzVMOSDisk -VM $vm -ManagedDiskId $osDisk.Id -StorageAccountType Standard_LRS -DiskSizeInGB 128 -CreateOption Attach -Windows

# Create the VM
$createdVmInfo = New-AzVM -ResourceGroupName $resourceGroupName -Location $location -VM $vm
$createdVmInfo


## Connect to the VM

In [ ]:
Get-AzRemoteDesktopFile -ResourceGroupName $resourceGroupName -Name $vmName -Launch

### Save the RDP file

In [ ]:
# Get-AzRemoteDesktopFile -ResourceGroupName $resourceGroupName -Name $vmName -LocalPath $rdpFilePath
# Invoke-Item $localPath